**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import (mean_squared_error,
                             mean_absolute_error)
from sklearn.linear_model import LinearRegression
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
from class_utils import error_histogram
import numpy as np
import pandas as pd

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Lineárna regresia

V predchádzajúcom príklade sme ukázali, ako sa dá pomocou optimalizácie realizovať jednoduchá regresia. Optimalizáciu sme realizovali iteratívne: pomocou gradientného prístupu. Existuje však jedna špeciálna skupina modelov, kde sa optimalizácia nemusí robiť iteratívne a optimálne parametre sa dajú určiť priamym výpočtom. Ide o lineárne modely s kvadratickou chybovou funkciou a optimálne parametre sa dajú vyrátať pomocou tzv. **metódy najmenších štvorcov**  (názov je z toho, že sa minimalizujú druhé mocniny chýb; angl. ordinary least squares). Výsledkom je prístup umožňujúci realizovať **lineárnu regresiu** .

S lineárnou regresiou sa v praxi dá stretnúť často: kedykoľvek treba nejaké body preložiž priamkou. Keďže teórii lineárnej regresie sa venujeme osobitne, tu uvedieme iba krátky praktický návod.

### Jednoduchý príklad: syntetická dátová množina

Vytvoríme si syntetickú dátovú množinu. Body budú z priamky, ale s gaussovským šumom, t.j.:
\begin{equation}
y_i = a x_i + c + \mathcal{N}(\mu, \sigma^2).
\end{equation}

Gaussovské (normálne) rozdelenie získame pomocou funkcie `np.random.normal`.



In [ ]:
df = pd.DataFrame()
df['x'] = np.arange(0, 1, 0.0025)
df['y'] = df['x'] + np.random.normal(0, 0.2, df['x'].shape)

df.head()

### Predspracovanie dát

Rozdelenie a predspracovanie dátovej množiny je úplne analogické ako v predchádzajúcich prípadoch, kód nasledujúcej bunky je teda skrytý.



In [ ]:
#@title -- Data Loading and Preprocessing; X_train, Y_train, X_test, Y_test -- { display-mode: "form" }
kbins = KBinsDiscretizer(6, encode='ordinal')
y_stratify = kbins.fit_transform(df[['y']])

df_train, df_test = train_test_split(df, stratify=y_stratify,
                                 test_size=0.3, random_state=4)

categorical_inputs = []
numeric_inputs = ['x']
output = ['y']

input_preproc = make_column_transformer(
    (make_pipeline(
        SimpleImputer(strategy="most_frequent"),
        OrdinalEncoder()),
     categorical_inputs),
    
    (make_pipeline(
        SimpleImputer(),
        StandardScaler()),
     numeric_inputs)
)

X_train = input_preproc.fit_transform(df_train[categorical_inputs+numeric_inputs])
Y_train = df_train[output]

X_test = input_preproc.transform(df_test[categorical_inputs+numeric_inputs])
Y_test = df_test[output]

plt.figure()
plt.scatter(X_train, Y_train, label="training data")
plt.scatter(X_test, Y_test, label="testing data")
plt.grid(ls='--')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.savefig("output/linreg_data.pdf", bbox_inches='tight', pad_inches=0)

#### Ladenie modelu

Vytvoríme regresný model a nastavíme jeho parametre pomocou funkcie `fit`.



In [ ]:
model = LinearRegression()
model = model.fit(X_train, Y_train)

#### Testovanie modelu

Ak chceme získať predikcie pre nejaké (tie isté alebo iné) vstupné dáta, použijeme funkciu `predict`. Funkcie `fit` a `predict` tvoria štandardné rozhranie u všetkých modelov z balíčka `sklearn`. Zvyšná časť fázy testovania bude vyzerať úplne obdobne ako v predchádzajúcom príklade.



In [ ]:
y_test = model.predict(X_test)

In [ ]:
#@title -- Testing -- { display-mode: "form" }

# we compute and display the MSE and the MAE
mse = mean_squared_error(Y_test, y_test)
print("MSE = {}".format(mse))

mae = mean_absolute_error(Y_test, y_test)
print("MAE = {}".format(mae))

plt.figure(figsize=(8, 6))
error_histogram(Y_test, y_test, Y_fit_scaling=Y_train)
plt.savefig("output/error_output_histogram.pdf", bbox_inches='tight', pad_inches=0)

# we visualize the regression line
x_min = min(np.min(X_train), np.min(X_test))
x_max = max(np.max(X_train), np.max(X_test))

xx = np.linspace(x_min, x_max, 250).reshape((-1, 1))
yy = model.predict(xx.reshape([-1, 1]))

plt.figure()
plt.scatter(X_train, Y_train, label="training data", s=15)
plt.scatter(X_test, Y_test, label="testing data", s=15)
plt.plot(xx, yy, 'k', linewidth='5', label="regressive model")

plt.grid(ls='--')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.savefig("output/linreg_line.pdf", bbox_inches='tight', pad_inches=0)